In [6]:
#!/usr/bin/env python3

import glob
import re
import nibabel as nib
import numpy as np
import os
import argparse
import h5py
from sklearn import linear_model
from scipy import stats,special
from subprocess import call
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import nibabel as nb
import nilearn
from nilearn import plotting, surface, datasets
import sys
import tqdm
setpath = '/home/msachs/music_emo_events/'
if setpath not in sys.path:
    sys.path.append(setpath)
from settings import *
import importlib

script_dir = '/home/msachs/my_libraries/'
if script_dir not in sys.path:
    sys.path.append(script_dir)
import hmm_functions
from hmm_functions import groupperm,runhmm, withinacross, match_z, entropy_p, bounds2events
#reload(hmm_functions)

#np.seterr(divide='ignore', invalid='ignore') ignore divide ny NAN

removesubs = ['sub-musevent05','sub-musevent06','sub-musevent80']
subs = [elem for elem in subs if elem.split('/')[5] not in removesubs] #if you want the cut ones

print('Num subs',len(subs), 'number of events',len(emo_offsets_tr['a1']))
print(volspace)
print(nilearn.__version__)

acompcor = 'acompcor'
print(acompcor)

Num subs 28 number of events 16
MNI152NLin2009cAsym
0.8.0
acompcor


## Calculate the autocorrelation in an ROI

In [ ]:
lag = 10
for roinum,roiname in enumerate(surfrois):
    print(roiname)
    data = dd.io.load(parcelpath + roiname)
    for pi,piecename in enumerate(pieces[0:4]):
        submean_data = data[piece.lower()]['D'].mean(axis = 0) #mean across people
         
        #getting autocorrelation
        autocor = np.zeros(lag)
        autocor[0] = 1
        for l in range(1,lag):
            corrs = np.zeros(submean_data.shape[0]-l)
            for t in range(submean_data.shape[0]-l):
                corrs[t] = pearsonr(submean_data[t],submean_data[t+l])[0]
            autocor[l] = np.mean(corrs)

        #plot results
        print('Graphing autocorrelation function for',roiname,piecename)
        plt.figure(figsize=(10,5))
        x = np.arange(0,lag)
        plt.plot(x,autocor)
        plt.xlim([0,lag])
        plt.ylim([0,1])
        plt.title(roiname)
        plt.show()

## Within vs. Across Correlations + HMM Matching - Cortical ROIS

In [ ]:
nPerm = 1000
window_size = 4


outpvals = {p:np.zeros((len(surfrois))) for p in pieces[0:4]}
for ri, roi_short in enumerate(surfrois):
    print(ri,roi_short)
#     if roi_short in subcortrois:
#         space = 'subcort'
#         outh5 = '%shmm_%s_%s_15over_%dss_%s_zscore.h5' %(subcortpath,task,roiname,groupnum,group)
#     else:
#         space = 'cort'
#         outh5 = '%shmm_%s_%s_15over_%dss_%s.h5' %(surfpath,task,roiname[:-3],groupnum,group)
        
    data = dd.io.load(parcelpath + roi_short)
    hmm_output = {}
    
    for pi,piecename in enumerate(pieces[0:4]):
        hmm_output[piecename] = {}
        piece = piecename.split('_')[1]
        #events = emo_events_tr[piece.lower()]
        events = emo_offsets_tr_lag[piece.lower()]
        bounds = events[:-1] #don't want the last one
        k = len(bounds)
        submean_data = data[piece.lower()]['D'].mean(axis = 0) #mean across people
        ntr = submean_data.shape[1]
        
        #run HMM
        hmm_output[piecename]['hmm_segments'],hmm_output[piecename]['bestfit_events'],hmm_output[piecename]['bestfit_bounds'],\
            events_entrop,hmm_diff, ll, ll_null,hmm_output[piecename]['pattern'] = runhmm(submean_data.T,k+1,ntr) #changed this on 03-30-21
        hmm_output[piecename]['ll_diff'] = ll-ll_null
        entropy_perm = entropy_p(events_entrop,bounds,ntr,nPerm)
        hmm_output[piecename]['entropy_zstat'] = (entropy_perm[0] - entropy_perm[1:].mean())/entropy_perm[1:].std()
        hmm_output[piecename]['entropy_pval'] = stats.norm.sf((hmm_output[piecename]['entropy_zstat'])) #one-sided
        outpvals[piecename][ri] = hmm_output[piecename]['entropy_pval']
        print(submean_data.shape, pi, piecename, k,'Entropy match',\
              round(hmm_output[piecename]['entropy_zstat'],3),round(hmm_output[piecename]['entropy_pval'],3))
        
#         #Within vs across
#         #print('Running within v across...')
#         hmm_output[piecename]['withacross_corrs'],hmm_output[piecename]['withacross_zstat'],_,_ = withinacross(bounds,submean_data.T,nPerm,ntr,window_size)
#         hmm_output[piecename]['withinvacross_pval'] = utils.p_from_null(hmm_output[piecename]['withacross_corrs'][0], \
#                                                                         hmm_output[piecename]['withacross_corrs'][1:nPerm+1], side = 'right',exact=False, axis=None) #side='right'
#         withinvacross_zstat = (hmm_output[piecename]['withacross_corrs'][0] - \
#                                hmm_output[piecename]['withacross_corrs'][1:].mean())/hmm_output[piecename]['withacross_corrs'][1:].std()
#         hmm_output[piecename]['withacross_pval_fromz'] = stats.norm.sf((withinvacross_zstat)) #one-sided
#         print(submean_data.shape, pi, piecename, k,'Within v across',\
#               round(withinvacross_zstat,3),round(hmm_output[piecename]['withinvacross_pval'],3),\
#               round(hmm_output[piecename]['withacross_pval_fromz'],3))
        
#         outpvals[piecename][ri] = hmm_output[piecename]['withacross_pval_fromz']
    
for pi,piecename in enumerate(pieces[0:4]):
    q = multipletests(outpvals[piecename], method='fdr_by')[1]
    print(outpvals[piecename].min(),q.min())
    print(surfrois[np.where(outpvals[piecename] == outpvals[piecename].min())[0][0]])
    print('Num that survive threshold:',len(np.where(rej)[0]))
    print(np.array(surfrois)[rej])

        

0 LH_DefaultA_IPL_1acompcor.h5
(501, 597) 0 051921_A1_master.mp4 15 Entropy match 0.738 0.23
(501, 597) 1 051921_A2_master.mp4 15 Entropy match 1.23 0.109
(501, 597) 2 051921_B1_master.mp4 15 Entropy match -0.984 0.837
(501, 597) 3 051921_B2_master.mp4 15 Entropy match -1.295 0.902
1 LH_DefaultA_PCC_1acompcor.h5
(466, 597) 0 051921_A1_master.mp4 15 Entropy match 1.192 0.117
(466, 597) 1 051921_A2_master.mp4 15 Entropy match 0.907 0.182
(466, 597) 2 051921_B1_master.mp4 15 Entropy match 0.241 0.405
(466, 597) 3 051921_B2_master.mp4 15 Entropy match -0.838 0.799
2 LH_DefaultA_PCC_2acompcor.h5
(238, 597) 0 051921_A1_master.mp4 15 Entropy match 1.94 0.026
(238, 597) 1 051921_A2_master.mp4 15 Entropy match 0.184 0.427
(238, 597) 2 051921_B1_master.mp4 15 Entropy match -2.79 0.997
(238, 597) 3 051921_B2_master.mp4 15 Entropy match -0.722 0.765
3 LH_DefaultA_PCC_3acompcor.h5
(331, 597) 0 051921_A1_master.mp4 15 Entropy match 0.302 0.381
(331, 597) 1 051921_A2_master.mp4 15 Entropy match 0.602